## Question 1

In [ ]:
import numpy as np
import pandas as pd
import requests
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from fastembed.embedding import TextEmbedding
import tqdm as notebook_tqdm

In [4]:
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\User\Miniconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Py

In [5]:
query = ["I just discovered the course. Can I join now?"]
embedding = list(embedder.embed(query))[0]

In [ ]:
print(f"Shape: {embedding.shape}")       
print(f"Min value: {np.min(embedding)}") 
print(f"Norm: {np.linalg.norm(embedding)}")  
print(f"Cosine similarity with self: {np.dot(embedding, embedding)}")  

Shape: (512,)
Min value: -0.11726374368207196
Norm: 1.0
Cosine similarity with self: 1.0


## Question 2

In [ ]:
# Embed a new document and calculate the cosine similarity with the query embedding
doc = ["Can I still join the course after the start date?"]
doc_vec = list(embedder.embed(doc))[0]
doc_vec

array([-5.45304216e-02, -7.83451890e-02,  3.13610220e-02,  2.34234659e-02,
       -3.06321531e-02, -5.51041666e-02,  2.24726443e-02,  1.50385846e-02,
       -1.81159603e-02,  1.80595452e-02, -4.80892935e-03, -7.57395912e-02,
       -4.87269909e-03, -9.07717914e-02,  4.83940834e-02, -4.26317335e-02,
       -2.61462336e-02, -4.48128965e-02, -2.06925942e-02, -3.79054456e-02,
       -3.30329167e-02, -8.02849907e-03,  1.18560345e-02, -3.83721145e-02,
        7.26378193e-02, -5.82811814e-02, -7.30890323e-02,  3.40911395e-02,
        5.60705315e-02,  5.02685143e-02, -5.99293168e-02,  5.33016965e-02,
       -1.79524626e-02,  4.17743639e-03, -2.45427843e-02,  1.32287952e-02,
        4.64178283e-02,  1.09963757e-02, -1.01500014e-02, -5.77526538e-03,
       -2.83983710e-04,  4.91496421e-02,  1.11214846e-01, -2.76306435e-02,
       -6.78923782e-02, -1.85928277e-02, -1.23963215e-01,  1.25745908e-02,
        1.65298563e-02,  7.22242041e-03, -2.77237445e-02,  5.18073796e-02,
       -5.98604691e-02,  

In [ ]:
# Calculate cosine similarity
similarity = np.dot(embedding, doc_vec)
print(f"Cosine similarity: {similarity:.2f}")

Cosine similarity: 0.90


## Question 3

In [11]:
# Prepare the documents for embedding
documents = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
     'question': 'Course - Can I still join the course after the start date?'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
     'question': 'Course - Can I follow the course after it finishes?'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
     'question': 'Course - When will the course start?'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
     'question': 'Course - What can I do before the course starts?'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
     'question': 'How can we contribute to the course?'},
]

In [12]:
# Embed documents' text fields
doc_texts = [doc["text"] for doc in documents]
doc_embeddings = list(embedder.embed(doc_texts))

# Convert to matrix for dot product
doc_matrix = np.vstack(doc_embeddings)

# Compute cosine similarity
similarities = doc_matrix @ embedding   # Dot product with the query embedding

# Get highest similarity index
best_index_q3 = np.argmax(similarities)
print(f"Q3: Highest similarity index: {best_index_q3}")

Q3: Highest similarity index: 1


## Question 4

In [13]:
# Create new combined field
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_embeddings = list(embedder.embed(full_texts))
full_matrix = np.vstack(full_embeddings)

# Compute similarity again
similarities_full = full_matrix @ embedding  # Dot product with the query embedding
best_index_q4 = np.argmax(similarities_full)
print(f"Q4: Highest similarity index: {best_index_q4}")

Q4: Highest similarity index: 0


## Question 5

In [22]:
# Fetch supported models and inspect key details
supported_models = (
    pd.DataFrame(TextEmbedding.list_supported_models())
    .sort_values("dim")  
    .drop(columns=["sources", "model_file", "additional_files"])  
    .reset_index(drop=True)
)

# Display the models
supported_models


,model,description,license,size_in_GB,dim,tasks
0,sentence-transformers/all-MiniLM-L6-v2,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,384,{}
1,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,384,{}
2,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,384,{}
3,snowflake/snowflake-arctic-embed-s,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,384,{}
4,sentence-transformers/paraphrase-multilingual-...,"Text embeddings, Unimodal (text), Multilingual...",apache-2.0,0.220,384,{}
5,BAAI/bge-small-en,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,384,{}
6,BAAI/bge-small-zh-v1.5,"Text embeddings, Unimodal (text), Chinese, 512...",mit,0.090,512,{}
7,Qdrant/clip-ViT-B-32-text,"Text embeddings, Multimodal (text&image), Engl...",mit,0.250,512,{}
8,jinaai/jina-embeddings-v2-small-en,"Text embeddings, Unimodal (text), English, 819...",apache-2.0,0.120,512,{}
9,nomic-ai/nomic-embed-text-v1,"Text embeddings, Multimodal (text, image), Eng...",apache-2.0,0.520,768,{}


In [14]:
embedder = TextEmbedding(model_name="BAAI/bge-small-en")

query = ["I just discovered the course. Can I join now?"]
embedding = list(embedder.embed(query))[0]

print(f"Dimensionality: {len(embedding)}")  

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\User\Miniconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|██████████| 5/5 [00:32<00

Dimensionality: 384


## Question 6

In [ ]:
# Fetch documents from the provided URL
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

documents = []

for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [ ]:
# Initialize in-memory Qdrant
client = QdrantClient(":memory:")
model = TextEmbedding(model_name="BAAI/bge-small-en")
collection_name = "ml_zoomcamp_faq"

# Create collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

C:\Users\User\AppData\Local\Temp\ipykernel_10912\86086124.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [25]:
# Prepare text and embeddings
payloads = []
vectors = []
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    embedding = list(model.embed([text]))[0]
    
    vectors.append(embedding)
    payloads.append(doc)

# Upload in bulk
client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(id=i, vector=vectors[i], payload=payloads[i])
        for i in range(len(documents))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
query = "I just discovered the course. Can I join now?"
query_vector = list(model.embed([query]))[0]

# Search
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1
)

# Highest score
score = results[0].score
print(f"Top score: {score:.2f}")

Top score: 0.87


C:\Users\User\AppData\Local\Temp\ipykernel_10912\3686366748.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
